### Lectura de los archivos csv y conversión a sql con creación de tablas

In [3]:
from Configuración_BD import engine
import pandas as pd

db = engine.connect()

df_alumnos = pd.read_csv("Bases de datos/Alumnos.csv",sep=",") # lee la base de datos "alumnos" que está en formato csv y lo guarda en la variable df
df_profesores = pd.read_csv("Bases de datos/Profesores.csv",sep=",") # lo mismo con "profesores"
df_cursos_profesores = pd.read_csv("Bases de datos/Cursos_profesores.csv",sep=",") # lo mismo con "cursos_profesores"


try:
	df_alumnos.to_sql("alumnos", db, if_exists="fail") # convierte el dataframe (df) de alumnos a sql y crea la tabla profes 
	df_profesores.to_sql("profesores", db, if_exists="fail") # lo mismo con "profesores"
	df_cursos_profesores.to_sql("cursos_profesores", db, if_exists="fail") # lo mismo con "cursos_profesores" 

except ValueError as vx:
	print(vx)
except Exception as ex:
	print (ex)
else:
	print("Tablas creadas con éxito")
finally:	
	db.close()

Table 'alumnos' already exists.


### Lectura de las tablas y conversión del dataframe a una lista de diccionarios (dataset)

In [2]:
df= pd.read_sql("select * from alumnos", engine) # lee la tabla alumnos (sql) y lo guarda en la variable df
dataset_alumnos=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset_alumnos)

db.close()

In [3]:
df= pd.read_sql("select * from profesores", engine) # lee la tabla profesores (sql) y lo guarda en la variable df
dataset_profesores=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset_profesores)


db.close()

In [4]:
df= pd.read_sql("select * from cursos_profesores", engine) # lee la tabla cursos_profesores (sql) y lo guarda en la variable df
dataset_cursos_profesores=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset_cursos_profesores)

db.close()

#### Eliminando el campo "index"

In [5]:
def eliminar_index(dataset):
    for x in dataset:
        del x["index"]   
                
eliminar_index(dataset_alumnos)
eliminar_index(dataset_profesores)

#### Extraer valores únicos de "gender" y traducirlos al español

In [6]:
def traduccion_gender(dataset):
    lista=[]
    for x in dataset:
        if x["gender"] not in lista:
            lista.append(x["gender"])    

    for x in dataset:
        if x["gender"]=="Female":
            x["gender"]="Femenino"
        elif x["gender"]=="Male":
            x["gender"]="Masculino"
        elif x["gender"]=="Non-binary":
            x["gender"]="No-binario"
        elif x["gender"]=="Polygender":
            x["gender"]="Poligénero"
        elif x["gender"]=="Genderqueer":
            x["gender"]="Género queer"
        elif x["gender"]=="Agender":
            x["gender"]="Agénero"
        elif x["gender"]=="Bigender":
            x["gender"]="Bigénero"
        elif x["gender"]=="Genderfluid":
            x["gender"]="Género fluido"   

traduccion_gender(dataset_profesores)
traduccion_gender(dataset_alumnos)

#### Convertir los campos personal_id e instructor a número, eliminando los puntos

In [8]:
def conversion_dni_numero(dataset):
    for x in dataset:
        if "personal_id" in x:                   
            j=x["personal_id"].replace('.','')
            x["personal_id"]=j
        if "instructor" in x:
            j=x["instructor"].replace('.','')
            x["instructor"]=j    
    
conversion_dni_numero(dataset_cursos_profesores)
conversion_dni_numero(dataset_alumnos)
conversion_dni_numero(dataset_profesores)

#### Eliminar registros repetidos en su totalidad o que tienen un campo repetido 

In [14]:

def eliminar_registros_repetidos_entotalidad(dataset):
    global nuevo_dataset  # para que nuevo_dataset exista por fuera de la función   
    lista_unica = set()   # conjunto con los registros únicos pero en forma de tuplas
    nuevo_dataset = []    # nuevo dataset con los registros únicos en forma de diccionarios
    repetidos = []  # lista con los registros eliminados

    for j in dataset:
        t = tuple(j.items()) # convierte a la lista de tuplas de los pares clave-valor de cada registro en una tupla de tuplas
        if t not in lista_unica: # si ese registro en forma de tupla de tuplas no está en el set lo agrega y a su 
                                 # vez agrega el registro en forma de diccionario a la lista
            lista_unica.add(t)
            nuevo_dataset.append(j)
        else:
            repetidos.append(j)      
        

def eliminar_registros_repetidos_uncampo(dataset,campo):
    valores_unicos = [] # crea una lista con todos los valores únicos del campo seleccionado
    registros_repetidos = [] # crea una lista con todos los registros en los cuales en el campo seleccionado tengan un valor repetido
    for x in dataset:
        for i,j in x.items():      
            if i==campo:          
                if j not in valores_unicos:        
                    valores_unicos.append(j)              
                else:                         
                    registros_repetidos.append(x)        
    for x in dataset:   # se elimina del dataset todos aquellos registros que se encuentren en la lista de registros_repetidos
        if x in registros_repetidos:
            del dataset[dataset.index(x)]   
    print(registros_repetidos) 



def buscar_registros_repetidos_uncampo(dataset,campo): # esta función es por si no se quiere eliminar aquellos registros con 
                                                       # campos repetidos, sino que solamente se los quiere encontrar para quizás modificar el 
                                                       # campo antes que borrar todo el registro
    valores_unicos = [] 
    registros_repetidos = [] 
    for x in dataset:
        for i,j in x.items():      
            if i==campo:          
                if j not in valores_unicos:        
                    valores_unicos.append(j)              
                else:                         
                    registros_repetidos.append(x)     
    print(registros_repetidos)
    





#### Para crear las tablas del archivo tablas

In [5]:
from Tablas import Base,engine,Profesores,Alumnos,Carreras,Profesores_Carreras,Alumnos_Carreras, Facultades,Ramas,Campus,Ubicacion,Provincias,Ciudades,Municipios,Genero
db = engine.connect()
Base.metadata.create_all(engine)

#### Insertar datos en tablas

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from Tablas import Base,engine,Profesores,Alumnos,Carreras,Profesores_Carreras,Alumnos_Carreras, Facultades,Ramas,Campus,Provincias,Ciudades,Municipios,Ubicacion,Genero

Session = sessionmaker(bind = engine)
session =  Session()

dataset_facultades=[] 
dataset_campus=[] 
dataset_ramas=[] 
dataset_genero=[] 
dataset_ubicacion=[] 
dataset_profesores2=[] 
dataset_alumnos2=[] 
dataset_carreras=[] 
dataset_profesores_carreras=[] 
dataset_alumnos_carreras=[]
dataset_ciudades=[] 
dataset_municipios=[] 
dataset_provincias=[] 


def generar_dataset(dataset_definitivo,dataset_original,campo,clave): #para generar los dataset de las tablas facultades, campus, ramas y genero
    valores_unicos=[]      
    for x in dataset_original:
        if x[campo] not in valores_unicos:
            valores_unicos.append(x[campo])
            dataset_definitivo.append({clave:(x[campo])})        

generar_dataset(dataset_facultades,dataset_cursos_profesores,"institute","nombre")
generar_dataset(dataset_ramas,dataset_cursos_profesores,"branch","nombre")
generar_dataset(dataset_campus,dataset_cursos_profesores,"campus","nombre")
generar_dataset(dataset_genero,dataset_alumnos,"gender","tipo")

def generar_dataset_c_p_m(dataset_definitivo,campo): #para generar los dataset de las tablas ciudades, provincias y municipios
    
    valores_unicos=[]

    for x in dataset_alumnos:        
            if x[campo] not in valores_unicos:
                valores_unicos.append(x[campo])
                dataset_definitivo.append({"nombre":(x[campo])})  
    for x in dataset_profesores:        
            if x[campo] not in valores_unicos:
                valores_unicos.append(x[campo])
                dataset_definitivo.append({"nombre":(x[campo])})       
    
generar_dataset_c_p_m(dataset_ciudades,"city")
generar_dataset_c_p_m(dataset_municipios,"town")
generar_dataset_c_p_m(dataset_provincias,"state")

def generar_dataset_ubicacion(): 
    
    for x in dataset_alumnos:
        ciudades_id=session.query(Ciudades).filter(Ciudades.nombre==x["city"]) # traigo todos los registros de la tabla ciudades cuyo campo nombre (el valor) sea igual al valor del campo "city" del dataset_alumnos
        municipios_id=session.query(Municipios).filter(Municipios.nombre==x["town"]) # lo mismo pero con el campo "town" (municipio)
        provincias_id=session.query(Provincias).filter(Provincias.nombre==x["state"]) # lo mismo pero con el campo "state" (provincia)
        for row in ciudades_id: # recorro cada uno de los campos de los registros obtenidos anteriormente
            dataset_ubicacion.append({"ciudades_id":row.id}) #agrego al dataset un diccionario cuyo valor se corresponde con el campo id de row
        for row in municipios_id: 
            dataset_ubicacion[dataset_alumnos.index(x)].update({"municipios_id":row.id}) # la misma operación que la anterior para el campo municipios, variando que en vez de agregar un nuevo diccionario modifico el ya existento agregándole un nuevo par clave-valor
        for row in provincias_id:
            dataset_ubicacion[dataset_alumnos.index(x)].update({"provincias_id":row.id})
            
    for x in dataset_profesores: # repetimos todo lo hecho anteriormente para el dataset_profesores, ya que hay ciudades, municipios y provincias exclusivos de este dataset que no están en el de alumnos
        ciudades_id=session.query(Ciudades).filter(Ciudades.nombre==x["city"])
        municipios_id=session.query(Municipios).filter(Municipios.nombre==x["town"])
        provincias_id=session.query(Provincias).filter(Provincias.nombre==x["state"])
        for row in ciudades_id:
            dataset_ubicacion.append({"ciudades_id":row.id})
        for row in municipios_id:
            dataset_ubicacion[dataset_profesores.index(x)+1000].update({"municipios_id":row.id}) # el +1000 en el index es para continuar desde el último registro del paso anterior, que inserta 1000 registros
        for row in provincias_id:
            dataset_ubicacion[dataset_profesores.index(x)+1000].update({"provincias_id":row.id})

   
generar_dataset_ubicacion()

#Luego procedo a eliminar los registros repetidos del dataset_ubicacion con la función eliminar_registros_repetidos_entotalidad #

eliminar_registros_repetidos_entotalidad(dataset_ubicacion)
dataset_ubicacion=nuevo_dataset # Acá realizo una copia del nuevo_dataset, generado con la función, para sobrescribir los registros de dataset_ubicacion con el fin de hacer efectivos los cambios realizados con la función, de otra forma dataset_ubicacion permance sin alterar al ser una variable definida por fuera de la función

def generar_dataset_carreras():           
    for x in dataset_cursos_profesores:       
        dataset_carreras.append({"nombre":x["program"]})    
        ramas_id=session.query(Ramas).filter(Ramas.nombre==x["branch"]) # traigo todos los registros de la tabla ciudades cuyo campo nombre (el valor) sea igual al valor del campo "branch" del dataset_cursos_profesores
        facultades_id=session.query(Facultades).filter(Facultades.nombre==x["institute"]) # lo mismo pero con el campo "institute" (municipio)
        campus_id=session.query(Campus).filter(Campus.nombre==x["campus"]) # lo mismo pero con el campo "campus" (provincia)
        for row in ramas_id: # recorro cada uno de los campos de los registros obtenidos anteriormente
            dataset_carreras[dataset_cursos_profesores.index(x)].update({"ramas_id":row.id}) #agrego al dataset un diccionario cuyo valor se corresponde con el campo id de row
        for row in facultades_id: 
            dataset_carreras[dataset_cursos_profesores.index(x)].update({"facultades_id":row.id}) # la misma operación que la anterior para el campo municipios, variando que en vez de agregar un nuevo diccionario modifico el ya existento agregándole un nuevo par clave-valor
        for row in campus_id:
            dataset_carreras[dataset_cursos_profesores.index(x)].update({"campus_id":row.id})

generar_dataset_carreras()
eliminar_registros_repetidos_entotalidad(dataset_carreras)
dataset_carreras=nuevo_dataset


def generar_dataset_alumnos_profesores(dataset_original,dataset_definitivo):  # Para generar los dataset de las tablas alumnos y profesores         
    for x in dataset_original:       
        dataset_definitivo.append({"dni":x["personal_id"]}) 
        dataset_definitivo[dataset_original.index(x)].update({"nombre":x["first_name"]})
        dataset_definitivo[dataset_original.index(x)].update({"apellido":x["last_name"]})
        dataset_definitivo[dataset_original.index(x)].update({"email":x["email"]})
        dataset_definitivo[dataset_original.index(x)].update({"pais":x["country"]})
        dataset_definitivo[dataset_original.index(x)].update({"fecha_nacimiento":x["birthdate"]})

        ciudad_alumno=session.query(Ciudades).filter(Ciudades.nombre==x["city"])
        for row in ciudad_alumno:
            id_ciudad_alumno=row.id
        municipio_alumno=session.query(Municipios).filter(Ciudades.nombre==x["town"])
        for row in municipio_alumno:
            id_municipio_alumno=row.id
        provincia_alumno=session.query(Provincias).filter(Ciudades.nombre==x["state"])
        for row in provincia_alumno:
            id_provincia_alumno=row.id
        ubicacion_id=session.query(Ubicacion).filter(Ubicacion.ciudades_id==id_ciudad_alumno and Ubicacion.municipios_id==id_municipio_alumno and Ubicacion.provincias_id==id_provincia_alumno) # traigo todos los registros de la tabla ciudades cuyo campo nombre (el valor) sea igual al valor del campo "branch" del dataset_cursos_profesores
        for row in ubicacion_id: # recorro cada uno de los registros obtenidos anteriormente
            dataset_definitivo[dataset_original.index(x)].update({"ubicacion_id":row.id}) #agrego al dataset un diccionario cuyo valor se corresponde con el campo id de row
        # campo genero_id
        genero_id=session.query(Genero).filter(Genero.tipo==x["gender"])
        for row in genero_id:
            dataset_definitivo[dataset_original.index(x)].update({"genero_id":row.id})

generar_dataset_alumnos_profesores(dataset_profesores,dataset_profesores2)
generar_dataset_alumnos_profesores(dataset_alumnos,dataset_alumnos2)



def generar_dataset_carreras_profesores():
    for x in dataset_cursos_profesores:
        profe=session.query(Profesores).filter(Profesores.dni==x["instructor"])
        carrera=session.query(Carreras).filter(Carreras.nombre==x["program"])
        for row in profe:            
            profe_id=row.id
        for row in carrera:
            carrera_id=row.id
        dataset_profesores_carreras.append({"profesor_id":profe_id,"carrera_id":carrera_id})
        
generar_dataset_carreras_profesores()

def generar_dataset_carreras_alumnos():
    for x in dataset_alumnos:
        alumno=session.query(Alumnos).filter(Alumnos.dni==x["personal_id"])
        carrera=session.query(Carreras).filter(Carreras.nombre==x["program"])
        for row in alumno:            
            alumno_id=row.id
        for row in carrera:
            carrera_id=row.id
        dataset_alumnos_carreras.append({"alumno_id":alumno_id,"carrera_id":carrera_id})

generar_dataset_carreras_alumnos()

def insertar_datos(dataset,tabla): # método para insertar los datos contenidos en cada dataset a su tabla correspondiente
    lista = []
    for x in dataset:
        lista.append(tabla(**x))
    session.add_all(lista)
    session.commit()
    session.close()

   
insertar_datos(dataset_ramas,Ramas)
insertar_datos(dataset_facultades,Facultades)
insertar_datos(dataset_campus,Campus)
insertar_datos(dataset_genero,Genero)
insertar_datos(dataset_ciudades,Ciudades)
insertar_datos(dataset_municipios,Municipios)
insertar_datos(dataset_provincias,Provincias)
insertar_datos(dataset_ubicacion,Ubicacion) 
insertar_datos(dataset_carreras,Carreras)
insertar_datos(dataset_alumnos2,Alumnos)
insertar_datos(dataset_profesores2,Profesores)
insertar_datos(dataset_alumnos_carreras,Alumnos_Carreras)
insertar_datos(dataset_profesores_carreras,Profesores_Carreras)



#### Exportar a csv

In [11]:
df= pd.read_sql("select * from alumnos", engine) # primero obtenemos el dataframe a partir de mysql
df.to_csv("asd.csv",index=False, encoding="utf-8") # después lo exportamos al formato csv, 
                                                   # eliminando el index que se agrega automáticamente 
                                                   # y codificándolo en utf-8 para su lectura en cualquier sistema 
                                                   # operativo

#### Reportes


La aplicación debe contar con la capacidad de generar al menos los siguientes reportes:  

* Listado con la cantidad de alumnos por carreas  
* Listado con la cantidad de profesores por carreas  
* Listado con el detalle de la cantidad de alumnos por provincia de origen y carrera que cursan  
* Distribución de alumnos por carrea y grupo etario (<20 años, entre 20 años y 30 años, >30 años)  
* Listado de profesores relacionado con cada alumno 

In [ ]:
x=session.query(Alumnos,Alumnos_Carreras,Carreras,Profesores_Carreras,Profesores).select_from(Alumnos).join(Alumnos_Carreras).join(Carreras).join(Profesores_Carreras).join(Profesores).all()
for alumno,alumnoxcarrera,carrera,profesorxcarrera,profesor in x:
    print(alumno.nombre,carrera.nombre,profesor.nombre)


In [10]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

Session = sessionmaker(bind = engine)
session =  Session()

x=session.query(Alumnos,Ubicacion,Ciudades,Municipios,Provincias).select_from(Alumnos).join(Ubicacion).join(Ciudades).join(Municipios).join(Provincias).all()
for j,i,h,y,p in x:
    print(j.nombre,h.nombre,y.nombre,p.nombre)

session.close()



Madelene POZO DEL MOLLE SAN FRANCISCO DEL MONTE DE ORO SAN LUIS
Rosaleen BALDECITO DE LA PAMPA AREA SIN GOBIERNO SAN LUIS
Judye LOS MOLLES AREA SIN GOBIERNO SAN LUIS
Zarla EL PARAISO AREA SIN GOBIERNO SAN LUIS
Melony TINTITACO LEANDRO N ALEM SAN LUIS
Ursulina CONCARAN CONCARAN SAN LUIS
Leila CORTADERAS CORTADERAS SAN LUIS
Camila VILLA SALLES JUSTO DARACT SAN LUIS
Korey CORONEL ALZOGARAY AREA SIN GOBIERNO SAN LUIS
Papagena LA SALVADORA AREA SIN GOBIERNO SAN LUIS
Farris COUNTRY CLUB LOS CALDENES AREA SIN GOBIERNO SAN LUIS
Sonnie BALDE DE AZCURRA AREA SIN GOBIERNO SAN LUIS
Ema LIBORIO LUNA AREA SIN GOBIERNO SAN LUIS
Cherish TALA VERDE LAS AGUADAS SAN LUIS
Corbet LA ROSADA AREA SIN GOBIERNO SAN LUIS
Beaufort SAN JERONIMO SAN JERONIMO SAN LUIS
Oralle NAHUEL MAPA AREA SIN GOBIERNO SAN LUIS
Charla EL VOLCAN EL VOLCAN SAN LUIS
Josiah EL CALDEN AREA SIN GOBIERNO SAN LUIS
Sheffy LAS CHACRAS JUANA KOSLAY SAN LUIS
Xymenes LAS CHACRAS JUANA KOSLAY SAN LUIS
Ingelbert LAS CHACRAS JUANA KOSLAY SAN LUI